In [1]:
import pandas as pd
import os

# 1) Buscar el último archivo en 'dataset/'
folder_dataset = 'dataset/Entrada/'
archivos_dataset = sorted([
    f for f in os.listdir(folder_dataset) 
    if f.startswith('df_completo_semana') and f.endswith('.csv')
])
archivo_mas_reciente = archivos_dataset[-1]
ruta_archivo = os.path.join(folder_dataset, archivo_mas_reciente)

# 2) Cargar el archivo
df = pd.read_csv(ruta_archivo)

# 3) Obtener la semana
semana = df["semana"].unique()
if len(semana) != 1:
    raise ValueError(f"Hay más de una semana en el archivo: {semana}")
n_semana = int(semana[0])

# 4) Limpiar
df = df[df['producto_costo_unitario'] != 0]
df = df[~((df['stock_actual'] == 0) & (df['inventario_ideal'] == 0))]

# 5) Calcular valores económicos
df['valor_stock_actual'] = df['stock_actual'] * df['producto_costo_unitario']
df['valor_stock_ideal'] = df['inventario_ideal'] * df['producto_costo_unitario']
df['diferencia_economica'] = df['valor_stock_ideal'] - df['valor_stock_actual']

# 6) Clasificación categoría
marcas_filtros = [
    "BALDWIN", "DONSSON", "AUT*PARTS", "RAMA", "RACOR USA", "RACOR BRASIL"
]

def clasificar_categoria(marca):
    marca = str(marca).upper().strip()
    if marca == "OTROS":
        return "Otros"
    elif marca in marcas_filtros:
        return "Filtro"
    else:
        return "Aceite/Lubricante"

df["categoria_producto"] = df["producto_marca"].apply(clasificar_categoria)

# 7) Guardar el resumen limpio individual
nombre_archivo = f'resumen_semana{n_semana}.xlsx'
ruta_resumen = f'inventarios_historicos/{nombre_archivo}'
df.to_excel(ruta_resumen, index=False)
print(f'✔ Resumen semana {n_semana} guardado como: {ruta_resumen}')

# 8) Crear histórico completo
# Buscar todos los archivos resumen existentes en inventarios_historicos/
folder_historico = 'inventarios_historicos/'
archivos_resumen = sorted([
    f for f in os.listdir(folder_historico) 
    if f.startswith('resumen_semana') and f.endswith('.xlsx')
])

# Leer y unir todos los resúmenes
df_historico = pd.DataFrame()
for archivo in archivos_resumen:
    ruta = os.path.join(folder_historico, archivo)
    df_parcial = pd.read_excel(ruta)
    df_historico = pd.concat([df_historico, df_parcial], ignore_index=True)

# Guardar el histórico completo
ruta_historico = os.path.join(folder_historico, 'historico_completo.xlsx')
df_historico.to_excel(ruta_historico, index=False)
print(f'✔ Histórico actualizado con {len(df_historico)} registros: {ruta_historico}')



✔ Resumen semana 34 guardado como: inventarios_historicos/resumen_semana34.xlsx
✔ Histórico actualizado con 86607 registros: inventarios_historicos/historico_completo.xlsx
